In [1]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import pandas as pd
import scipy as sc
from astropy.cosmology import Planck18  as cosmo # Planck 2018
import scipy.integrate as integrate
from astropy.cosmology import w0wzCDM
from astropy import units as u

In /home/nyavo/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/nyavo/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/nyavo/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/nyavo/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/nyavo/anaconda3/lib/python3.7/site-packages/matplotlib/

In [2]:
z_ = np.arange(0.01,2.03,0.05)
redshift = np.mean([z_[1],z_[2]])
c = 3*10**5 #km/s
d_L = np.mean(np.array(cosmo.luminosity_distance(np.mean([z_[1],z_[2]]))))
nwalkers, ndim = 10,1

In [3]:
df1 = pd.read_csv('run_1_samples.dat',delimiter=' ')
df2 = pd.read_csv('run_2_samples.dat',delimiter=' ')
df3 = pd.read_csv('run_3_samples.dat',delimiter=' ')
df4 = pd.read_csv('run_4_samples.dat',delimiter=' ')

In [4]:
d1 = sc.stats.gaussian_kde(df1['luminosity_distance'])
d2 = sc.stats.gaussian_kde(df2['luminosity_distance'])
d3 = sc.stats.gaussian_kde(df3['luminosity_distance'])
d4 = sc.stats.gaussian_kde(df4['luminosity_distance'])

In [5]:
pdf1=d1(df1["luminosity_distance"])
pdf2=d2(df2["luminosity_distance"])
pdf3=d3(df3["luminosity_distance"])
pdf4=d4(df4["luminosity_distance"])

In [6]:
print(len(pdf1),len(pdf2),len(pdf3),len(pdf4))

3553 3707 3525 3760


In [7]:
distribution = np.mean( pdf1[0:3524] * pdf2[0:3524] * pdf3[0:3524] *pdf4[0:3524] )

In [8]:
distribution

2.689790829485745e-08

In [9]:
def E(z):
    return np.sqrt ( (cosmo.Om0 * (1+z)**3) + (1-cosmo.Om0 ))
# result = integrate.quad(lambda z: E(z), 0, z) [0]

In [10]:
def log_likelihood(theta, z, df1 , pdf1):
    H0 = theta
    var = H0 * u.km/u.s/u.Mpc
    cosmow1 = w0wzCDM(H0=var, Om0=cosmo.Om0, Ode0=1.-cosmo.Om0, w0=-1, wz=0)   
    model = cosmow1.luminosity_distance(z)
    log_l = np.log(np.interp (model,df1,pdf1))
    return  log_l

In [11]:
def log_prior(theta):
    H0 = theta
    if 20 <= H0 <= 200:
        return 0.0
    return -np.inf

In [14]:
def log_probability(theta, z, df1, pdf1):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, z, df1, pdf1)

In [15]:
#emcee
pos = 40 + 0.1 * np.random.randn(10, 1)
nwalkers, ndim = 10,1

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(z, df1, pdf1))
sampler.run_mcmc(pos, 100, progress=True)
samples = sampler.chain

NameError: name 'z' is not defined

fig = plt.figure()
dim_name = [r'$H_0$']
for dim in range(ndim):
    ax = fig.add_subplot(ndim, 1, dim+1)
    for i in range(nwalkers):
        ax.plot(np.arange(100),
                samples[i, :, dim],
                ls='-',
                color='black',
                alpha=10./nwalkers)
    ax.set_ylabel(dim_name[dim], fontsize='large')
ax.set_xlabel('step', fontsize='large')